# Dual Mesh Construction

In [ ]:
import uxarray as ux
import warnings
import geocat.datafiles as geodf

warnings.filterwarnings("ignore")

In [ ]:
datafile = (
    geodf.get("netcdf_files/MPAS/FalkoJudt/dyamond_1/30km/x1.655362.grid_subset.nc"),
)

The dual mesh of a grid is where the face centers of all primal (original) grid become the nodes of the dual mesh, and the nodes of the primal mesh become the face centers of the primal mesh. Using UXarray we can construct this dual mesh, using `grid.compute_dual()`. `compute_dual()` takes on argument, `method`. Currently only global dual meshes are supported, local dual meshes are planned for a later date. `compute_dual()` returns a new grid object.

In [ ]:
grid = ux.open_grid(datafile[0])

In [ ]:
grid = grid.subset.nearest_neighbor((0, 0), 1000)

In [ ]:
dual = grid.compute_dual()

In [ ]:
dual.plot(backend="matplotlib", fig_size=300) * grid.plot(
    backend="matplotlib", fig_size=300
)